In [ ]:
from helper_fun import get_evaluator_info, evaluate_circ
from qcg.generators import gen_supremacy, gen_hwea
import numpy as np

for dimension in [[2,2],[1,5],[2,3],[1,7],[2,4],[3,3],[2,5],[3,4]]:
    i,j=dimension
    circ = gen_supremacy(i,j,8)
    device_name = 'ibmq_boeblingen'

    evaluator_info = get_evaluator_info(circ=circ,device_name=device_name,
        fields=['device','basis_gates','coupling_map','properties','initial_layout','num_shots','meas_filter'])
    hw_fc = evaluate_circ(circ=circ,backend='hardware',evaluator_info=evaluator_info)
    print('-'*50)

Generating 2x2, 1+8+1 supremacy circuit
Calculating measurement filter, 20-qubit calibration circuits * 16 * 2.048e+03 shots. 6.140e-01 seconds
Mitigation for 4 qubit circuit took 6.223e-03 seconds
--------------------------------------------------
Generating 1x5, 1+8+1 supremacy circuit
Calculating measurement filter, 20-qubit calibration circuits * 32 * 2.048e+03 shots. 7.807e-01 seconds
Mitigation for 5 qubit circuit took 1.563e-02 seconds
--------------------------------------------------
Generating 2x3, 1+8+1 supremacy circuit
Calculating measurement filter, 20-qubit calibration circuits * 64 * 3.072e+03 shots. 2.120e+00 seconds
Mitigation for 6 qubit circuit took 2.982e-02 seconds
--------------------------------------------------
Generating 1x7, 1+8+1 supremacy circuit
Calculating measurement filter, 20-qubit calibration circuits * 128 * 4.096e+03 shots. 4.685e+00 seconds
Mitigation for 7 qubit circuit took 1.380e-01 seconds
--------------------------------------------------
Gen